# ISIN Investigator

by David Cobbeldick, Laurance Day, Lewyse Lee, Simen Svnkerud

## Purpose
Bildt as a step in the quality assurance work within a larger pipline of work. 
This notebook took a bespoke input and outputted an indepth investigation into a given unit of analysis.
This notebook is reproduced as part of a portfolio of work and has thus been "anonymised"

In [1]:
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
import numpy as np
import itertools
import random
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

In [2]:
%matplotlib inline

In [3]:
sns.set(style="darkgrid")

In [4]:
LookUp = "4122025"

In [5]:
FA_Instruments = 'F511'

In [6]:
os.chdir('wdr')

In [7]:
Output_Path = r'\Data'
with open(r'Data\QA_Viewer.json') as f:
    data = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")
    

# Processing data
## Processing for Individual Securities

In [8]:
df = pd.DataFrame()

In [9]:
df.head()

""


In [10]:
Ref_Cols = ['Security_Id', 'Quarter']
List_Cols = ['UK_Owners', 
             'UK_Owners_Shares_Held', 
             'UK_Owners_Value_Held',
             'NonUK_Owners',
             'NonUK_Owners_Shares_Held',
             'NonUK_Owners_Value_Held']
Float_Cols = [x for x in list(df) if x not in Ref_Cols+List_Cols]

In [11]:
SIDs = list(set(df['Security_Id']))

#LookUp = random.choice(SIDs)

LookUp_df = df[df['Security_Id']==LookUp]\
                .sort_values('Quarter', ascending=True)\
                .set_index('Quarter', drop=True)
        
LookUp_df.head()

KeyError: 'Security_Id'

In [ ]:
print(LookUp)

In [ ]:
for i in Float_Cols:
    LookUp_df[i] = pd.to_numeric(LookUp_df[i], errors='coerce')
LookUp_df = LookUp_df.fillna(0)

Owners_df = LookUp_df[List_Cols].reindex()

In [ ]:
for i in list(Owners_df):
    Owners_df[i] = [x if x!=0\
                   else [] for x in Owners_df[i].tolist()]

In [ ]:
UK_Owners_Count = [len(x) for x in Owners_df['UK_Owners'].tolist()]
NonUK_Owners_Count = [len(x) for x in Owners_df['NonUK_Owners'].tolist()]

LookUp_df = LookUp_df[[x for x in list(LookUp_df)\
                       if x not in List_Cols+['Security_Id']]]

In [ ]:
LookUp_df['Issuer_Price'] = LookUp_df['Mkt_Cap']/LookUp_df['Shares_Outstanding']
LookUp_df['Total_Value_Held'] = LookUp_df['UK_Value_Held']+LookUp_df['NonUK_Value_Held']
LookUp_df['Total_Shares_Held'] = LookUp_df['UK_Shares_Held']+LookUp_df['NonUK_Shares_Held']
LookUp_df['Owner_Price'] = LookUp_df['Total_Value_Held']/LookUp_df['Total_Shares_Held']
LookUp_df['UK_Coverage'] = (LookUp_df['UK_Value_Held']/LookUp_df['Mkt_Cap'])*100
LookUp_df['NonUK_Coverage'] = (LookUp_df['NonUK_Value_Held']/LookUp_df['Mkt_Cap'])*100
LookUp_df['Total_Value_Coverage'] =  (LookUp_df['Total_Value_Held']/LookUp_df['Mkt_Cap'])*100
LookUp_df['Total_Shares_Coverage'] = (LookUp_df['Total_Shares_Held']/LookUp_df['Shares_Outstanding'])*100
LookUp_df['UK_Owners'] = UK_Owners_Count
LookUp_df['NonUK_Owners'] = NonUK_Owners_Count
LookUp_df['Total_Owners'] = LookUp_df['UK_Owners']+LookUp_df['NonUK_Owners']
LookUp_df['UK_Value_Held_Calc'] = LookUp_df['UK_Shares_Held']*LookUp_df['Issuer_Price']
LookUp_df['NonUK_Value_Held_Calc'] = LookUp_df['NonUK_Shares_Held']*LookUp_df['Issuer_Price']
LookUp_df['Total_Value_Held_Calc'] = LookUp_df['NonUK_Value_Held_Calc']+LookUp_df['UK_Value_Held_Calc']
   
LookUp_df = LookUp_df[['Issuer_Price',
                 'Owner_Price',
                 'Mkt_Cap',
                 'Shares_Outstanding',
                 'UK_Value_Held',
                 'NonUK_Value_Held',
                 'Total_Value_Held',
                 'UK_Shares_Held',
                 'NonUK_Shares_Held',
                 'Total_Shares_Held',
                 'UK_Coverage',
                 'NonUK_Coverage',
                 'Total_Value_Coverage',
                 'Total_Shares_Coverage',
                 'UK_Owners',
                 'NonUK_Owners',
                 'Total_Owners',
                 'UK_Value_Held_Calc',
                'NonUK_Value_Held_Calc',
                      'Total_Value_Held_Calc']]

In [ ]:
#=============================================================================
'''Owner Breakdowns'''
#=============================================================================

if sum(UK_Owners_Count)>0:

    UK_Owner_List = Owners_df['UK_Owners'].tolist()
    UK_Owner_List = [x for x in UK_Owner_List if len(x)>0]
    UK_Owner_List = [val for sublist in UK_Owner_List for val in sublist]
    UK_Owner_List = list(set(UK_Owner_List))
    UK_Owner_df = pd.DataFrame({'UK_Owners':UK_Owner_List})
    
    UK_Owners_SH = UK_Owner_df.reindex()
    UK_Owners_VH = UK_Owner_df.reindex()

    for i in Owners_df.index.tolist():
    
    # UK Shares Held per Owner DataFrame
        if len(Owners_df.at[i,'UK_Owners'])>0:
            UK_SH = pd.DataFrame({'UK_Owners':Owners_df.at[i,'UK_Owners'],\
                                  'SH':Owners_df.at[i,'UK_Owners_Shares_Held']})
            UK_SH = UK_SH.rename(columns = {'SH':i})
            UK_Owners_SH = UK_Owners_SH.merge(UK_SH, on='UK_Owners', how='left')\
            .drop_duplicates(subset='UK_Owners')
        elif len(Owners_df.at[i,'UK_Owners'])==0:
            UK_Owners_SH[i] = np.nan
     
    # UK Value Held per Owner DataFrame
        if len(Owners_df.at[i,'UK_Owners'])>0:
            UK_VH = pd.DataFrame({'UK_Owners':Owners_df.at[i,'UK_Owners'],\
                                  'VH':Owners_df.at[i,'UK_Owners_Value_Held']})
            UK_VH = UK_VH.rename(columns = {'VH':i})
            UK_Owners_VH = UK_Owners_VH.merge(UK_VH, on='UK_Owners', how='left')\
            .drop_duplicates(subset='UK_Owners')
        elif len(Owners_df.at[i,'UK_Owners'])==0:
            UK_Owners_VH[i] = np.nan
    
    UK_Owners_SH = UK_Owners_SH.set_index('UK_Owners', drop=True)               
    UK_Owners_SH = UK_Owners_SH.transpose()    
    UK_Owners_VH = UK_Owners_VH.set_index('UK_Owners', drop=True)
    UK_Owners_VH = UK_Owners_VH.transpose()
        
else:
    UK_Owners_SH = None
    UK_Owners_VH = None




In [ ]:
if sum(NonUK_Owners_Count)>0:

    NonUK_Owner_List = Owners_df['NonUK_Owners'].tolist()
    NonUK_Owner_List = [x for x in NonUK_Owner_List if len(x)>0]
    NonUK_Owner_List = [val for sublist in NonUK_Owner_List for val in sublist]
    UK_Owner_List = list(set(NonUK_Owner_List))
    NonUK_Owner_df = pd.DataFrame({'NonUK_Owners':NonUK_Owner_List})
    
    NonUK_Owners_SH = NonUK_Owner_df.reindex()
    NonUK_Owners_VH = NonUK_Owner_df.reindex()

    for i in Owners_df.index.tolist():
        
    # NonUK Shares Held per Owner DataFrame
        if len(Owners_df.at[i,'NonUK_Owners'])>0:
            NonUK_SH = pd.DataFrame({'NonUK_Owners':Owners_df.at[i,'NonUK_Owners'],\
                                  'SH':Owners_df.at[i,'NonUK_Owners_Shares_Held']})
            NonUK_SH = NonUK_SH.rename(columns = {'SH':i})
            NonUK_Owners_SH = NonUK_Owners_SH.merge(NonUK_SH, on='NonUK_Owners', how='left')\
            .drop_duplicates(subset='NonUK_Owners')
        elif len(Owners_df.at[i,'NonUK_Owners'])==0:
            NonUK_Owners_SH[i] = np.nan 

    # NonUK Shares Held per Owner DataFrame    
        if len(Owners_df.at[i,'NonUK_Owners'])>0:
            NonUK_VH = pd.DataFrame({'NonUK_Owners':Owners_df.at[i,'NonUK_Owners'],\
                                  'VH':Owners_df.at[i,'NonUK_Owners_Value_Held']})
            NonUK_VH = NonUK_VH.rename(columns = {'VH':i})
            NonUK_Owners_VH = NonUK_Owners_VH.merge(NonUK_VH, on='NonUK_Owners', how='left')\
            .drop_duplicates(subset='NonUK_Owners')
        elif len(Owners_df.at[i,'NonUK_Owners'])==0:
            NonUK_Owners_VH[i] = np.nan
    
    NonUK_Owners_SH = NonUK_Owners_SH.set_index('NonUK_Owners', drop=True)
    NonUK_Owners_SH = NonUK_Owners_SH.transpose()     
    NonUK_Owners_VH = NonUK_Owners_VH.set_index('NonUK_Owners', drop=True)
    NonUK_Owners_VH = NonUK_Owners_VH.transpose()
else:
    NonUK_Owners_SH = None
    NonUK_Owners_VH = None


In [ ]:
if len(UK_Owners_VH) >0:
    VH_plot1 = UK_Owners_VH.add_prefix('UK_')
if len(NonUK_Owners_VH)>0:
    VH_plot2 = NonUK_Owners_VH.add_prefix('Non_UK_')
    
VH_plot = VH_plot1.join(VH_plot2)

In [ ]:
if len(UK_Owners_SH) >0:
    SH_plot1 = UK_Owners_SH.add_prefix('UK_')

if len(NonUK_Owners_SH)>0:
    SH_plot2 = NonUK_Owners_SH.add_prefix('NonUK_')
    
SH_plot = SH_plot1.join(SH_plot2)

# Start of analysis

### Set graphing parameters

In [ ]:
#Time_Start = 
#Time_End = 

In [ ]:
VH_plot.head(20)

In [ ]:
plt.figure(1)
LookUp_df['Mkt_Cap'].plot.line(figsize=(20,5))
plt.legend(ncol=int(len(VH_plot.columns)/5))
plt.title('Total Market capitalisation by value')

plt.figure(2)
VH_plot.plot(kind='bar', stacked = True, color=sns.color_palette("spectral",len(VH_plot.columns), desat=.5),figsize = (20,5), legend = False)
#plt.legend(ncol=int(len(VH_plot.columns)/5))
plt.xticks(rotation=90)
plt.title('Security value owned broken down by Owner ID')
plt.show()

## Coverage by Value and shares

Bellow is a plot giving the ratio of known Issuance and ownership 

In [ ]:
plt.figure(3)
LookUp_df['Total_Value_Coverage'].plot(kind='line', c='r', figsize = (20,5))
LookUp_df['Total_Shares_Coverage'].plot(kind='line', figsize = (20,5))
plt.legend(ncol=int(len(VH_plot.columns)/5))
plt.title('Security Coverage  over time based on value and numer of Shares')
plt.show()

## Price comparison

Price over time, as collected from the EOD price history (Issuer_Price), and derived from Thomson reuters value_held/Shares_held.
Used for internal QA.

Issuer price is plotted here to provide visual for the change in price

In [ ]:
plt.figure(4)
LookUp_df['Issuer_Price'].plot(kind='Line', c='g', figsize = (20,5))
LookUp_df['Owner_Price'].plot(kind='Line', c='r', figsize = (20,5))
plt.title('Share price over time')
plt.legend()

## Market Cap, Shares Outstanding and Shares held

Visual presentation of the effect of price. Level of ownership by count and distribution of UK ROW Share oldership.

First plot is a reminder from earlier and plots the total market capitalisation of the security.
Secon plot gives total issuance in number shares, total UK owned Shares, Total ROW owned Shares, total known share ownership by numbers.
Third plot breaks down known ownership by owner, using number of shares held.

In [ ]:
plt.figure(5)
LookUp_df['Mkt_Cap'].plot(kind='line', figsize = (20,5))
plt.legend()
plt.title('Total Market capitalisation by value')

plt.figure(6)
LookUp_df['Shares_Outstanding'].plot(kind='line',c='g', figsize = (20,5))
LookUp_df['Total_Shares_Held'].plot(kind='line', c= 'c', figsize = (20,5))
plt.legend(ncol=int(len(VH_plot.columns)/5))
plt.title('Issuance and number of shares over time')

plt.figure(7)
SH_plot.plot(kind='bar', stacked = True, color=sns.color_palette("spectral",len(VH_plot.columns), desat=.5),figsize = (20,5), legend = False)
#plt.legend(ncol=int(len(SH_plot.columns)/5))
plt.xticks(rotation=90)
plt.title('Number of shares owend by individual owners over time')
plt.show()

## Mkt_cap and Value held (claculated vs TR provided)

Internal QA step. Ploting the total Value_held as given by TR and the Total Value_held as calculates using number of shares held and the price infroation data

In [ ]:
plt.figure()
LookUp_df['Mkt_Cap'].plot(kind='line', figsize=(20,5))
LookUp_df['Total_Value_Held'].plot(kind = 'line', c='r', figsize =(20,5))
LookUp_df['Total_Value_Held_Calc'].plot(kind = 'line', c='c', figsize =(20,5))
plt.legend(ncol=int(len(LookUp_df.columns)/5))
plt.title('Market Capitaisation by value and value held')
plt.show()

## Count of owners

In [ ]:
plt.figure()
LookUp_df['Total_Owners'].plot(kind='line', figsize = (20,5))
LookUp_df['UK_Owners'].plot(kind='line', c='c', figsize = (20,5))
LookUp_df['NonUK_Owners'].plot(kind='line',c= 'b', figsize = (20,5))
plt.legend(ncol=int(len(LookUp_df.columns)/5))
plt.title('Count of owners for the security')
plt.show()

## Distribution of owner type over time

# Per Security geographically distribution (UK and ROW)

## Distribution of Value Held by UK or ROW


In [ ]:
plot_df = LookUp_df[['UK_Value_Held_Calc',
                'NonUK_Value_Held_Calc']]
plt.figure()
LookUp_df['Mkt_Cap'].plot(kind = 'line', figsize=(20,5))
plt.legend(ncol=int(len(LookUp_df.columns)/5))
plt.title('Security Market capitalisation by value')
plot_df.plot(kind='bar', stacked = True, figsize=(20,5))
plt.legend(ncol=int(len(LookUp_df.columns)/5))
plt.title('value of security held by the UK and ROW')
plt.show()

## Distribution of number of shares held by UK and ROW

In [ ]:
plot_df = LookUp_df[['UK_Shares_Held',
                'NonUK_Shares_Held']]
plt.figure()
LookUp_df['Shares_Outstanding'].plot(kind = 'line', figsize=(20,5))
plt.legend(ncol=int(len(LookUp_df.columns)/5))
plt.title('Total shares outstanding')
plot_df.plot(kind='bar', stacked = True, figsize=(20,5))
plt.legend(ncol=int(len(LookUp_df.columns)/5))
plt.title('Shears held bu UK and Non UK Owners')
plt.show()

In [ ]:
#plt.figure(5)
#fig, axes = plt.subplots(nrows = 2, ncols = 1)
#LookUp_df['Shares_Outstanding'].plot(ax = axes[0], kind='line')
#LookUp_df['Issuer_Price'].plot(ax = axes[1],kind='Line')